# TODO 

* Сделай аттеншн

# Imports

In [1]:
import keras
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras'

In [1]:
#import keras
import os
import datasets
import numpy as np
import pandas as pd
import transformers
import sklearn.metrics
#import tensorflow as tf
import tqdm.notebook as tqdm
import sklearn.model_selection
import matplotlib.pyplot as plt

#pytorch

import torch
from torcheval.metrics import BLEUScore
#from torchmetrics.classification import MultilabelAUROC
#from torchsummary import summary

from sklearn.model_selection import train_test_split

/home/bob1ch/Рабочий стол/NN-NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = 'models'
MAX_TOKENS = 32
MAX_LENGTH = 32

# Dataset

Get the dataset from [here](https://tatoeba.org/en/downloads). Preferably use russian to english translations.

Use a custom tokenizer that can add bos and eos tokens (pass `add_special_tokens=True` when calling the tokenizer to add them).

In [3]:
class Tokenizer(transformers.GPT2Tokenizer):

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.bos_token_id, *token_ids_0, self.eos_token_id]

        return [self.bos_token_id, *token_ids_0, self.bos_token_id, *token_ids_1, self.eos_token_id]

In [4]:
tokenizer = Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'Tokenizer'.
/home/bob1ch/Рабочий стол/NN-NLP/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def to_tokenize(data):
    return tokenizer(data, max_length=MAX_TOKENS, truncation=True, padding=True)['input_ids']

Since the dataset is rather large, you can omit the validation dataset and just use a set of test sentences after the training.

Create a dataset that returns the following
* A pair of tensors `((None, L), (None, P))` -- input sequence of tokens and output sequence of tokens to be fed into decoder (this should start with the BOS token)
* A tensor `(None, P)` -- output sequence of tokens to be predicted (this should end with EOS token)
* A tensor `(None, P)` -- a masking tensor marking padded tokens with 0

In [6]:
data = pd.read_csv('seq2seq_dataset.tsv', 
            sep='\t', 
            on_bad_lines='skip',
            names=['id_1', 'rus', 'id_2', 'eng'])[['rus', 'eng']]#тут скип, потомучта какое-то говно возникало

X, y = to_tokenize(data['rus'].to_list()), to_tokenize(data['eng'].to_list())

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.7, shuffle=True)

In [8]:
np.array(X)[:, ::-1]

array([[50256,   141,   220, ...,   252,   140, 50256],
       [50256, 35072,   109, ...,   242,   140, 50256],
       [50256, 50256, 50256, ...,   250,   140, 50256],
       ...,
       [50256, 50256, 50256, ...,   250,   140, 50256],
       [50256, 50256, 50256, ...,   250,   140, 50256],
       [50256, 50256, 50256, ...,   252,   140, 50256]])

In [9]:
def pad_collate(batch):
    (inp, out_BOS, out_EOS, masking_PAD) = zip(*batch)
    inp = torch.nn.utils.rnn.pad_sequence(inp, batch_first=True, padding_value=tokenizer.eos_token_id)
    out_BOS = torch.nn.utils.rnn.pad_sequence(out_BOS, batch_first=True, padding_value=tokenizer.eos_token_id)
    out_EOS = torch.nn.utils.rnn.pad_sequence(out_EOS, batch_first=True, padding_value=tokenizer.eos_token_id)
    masking_PAD = torch.nn.utils.rnn.pad_sequence(masking_PAD, batch_first=True, padding_value=tokenizer.eos_token_id)

    return inp, out_BOS, out_EOS, masking_PAD

def get_dataloader(batch, X, y):
    X = np.array(X)[:, ::-1] #Торч поругался, что я не могу сделать отрицательный степ
    inp = torch.LongTensor(X.copy()).to('cuda') #а потом он ещё поругался, что в массиве у меня отрицательный страйд и попросил .copy()
    out_BOS = torch.LongTensor(y).to('cuda')[:, :-1]
    out_EOS = torch.LongTensor(y).to('cuda')[:, 1:]
    masking_PAD = (out_EOS != tokenizer.pad_token_id).to('cuda')
    #masking_PAD[:, 0] =  1 #BOS==EOS поэтому тут ручками доделал
    data = torch.utils.data.TensorDataset(inp,
                                          out_BOS,
                                          out_EOS,
                                          masking_PAD)
    return torch.utils.data.DataLoader(data, batch_size=batch, shuffle=True, collate_fn=pad_collate)

In [10]:
data = get_dataloader(64, X_train, y_train)
data_test = get_dataloader(64, X_test[:20], y_test[:20])

# Model

Create a model for training. The model should have two inputs: input sequence `(None, L)` and output sequence`(None, P)`. The model output is a single tensor `(None, P)` logits (or probabilities) of the next token predicted for each input one.

In [17]:
class Encoder(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, bidirectional: bool, name: str):
        super(Encoder, self).__init__()
        self.name = name
        self.units = units
        self.bidirectional = bidirectional
        
        self.embeddings = torch.nn.Embedding(n_tokens, units)
        self.rnn = torch.nn.GRU(units, units, num_layers=n_stacks, bidirectional=bidirectional, batch_first=True) #можно поменять

    def forward(self, text):
        embeds = self.embeddings(text)      
        out, h = self.rnn(embeds)

        return out, h

# class BahdanauAttention(torch.nn.Module):
#     def __init__(self, hidden_size):
#         super(BahdanauAttention, self).__init__()
#         self.Wa = torch.nn.Linear(hidden_size, hidden_size)
#         self.Ua = torch.nn.Linear(hidden_size, hidden_size)
#         self.Va = torch.nn.Linear(hidden_size, 1)

#     def forward(self, query, keys):
#         scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
#         scores = scores.squeeze(2).unsqueeze(1)

#         weights = F.softmax(scores, dim=-1)
#         context = torch.bmm(weights, keys)

#         return context, weights

#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-decoder
class Decoder(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, bidirectional: bool, name: str): 
        super(Decoder, self).__init__()
        self.name = name
        self.units = units
        #self.bidirectional = bidirectional
        
        self.embeddings = torch.nn.Embedding(n_tokens, units)
        # self.attention = BahdanauAttention(units)
        self.rnn = torch.nn.GRU(units, units , num_layers=n_stacks, batch_first=True)
        self.FC = torch.nn.Linear(units, n_tokens)

    def forward(self, enc_outs, enc_h, target=None):
        #enc_outs нужен чтобы батч узнать
        batch_size = enc_outs.size(0)
        dec_inp = torch.empty(batch_size, 1, dtype=torch.long, device='cuda').fill_(tokenizer.bos_token_id)
        dec_h, dec_out = enc_h, []
        
        #цикл генерации токенов, подумой можно ли остановить, если пришел паддинг
        out, dec_h = self.apply_rnn(dec_inp, dec_h, enc_outs)
        dec_out.append(out)
        for i in range(1, MAX_LENGTH-1):
            out, dec_h = self.apply_rnn(dec_inp, dec_h, enc_outs)
            dec_out.append(out)
            dec_inp = target[:, i].unsqueeze(1) if target is not None else out.argmax(dim=-1).detach()
            
            
        dec_out = torch.cat(dec_out, dim=1)
        return dec_out

    def apply_rnn(self, inp, h, enc_out):
        embeds = self.embeddings(inp)
        
        #query = h.permute(1, 0, 2)
        #context, attn_weights = self.attention(query, enc_out)
        
        #input_gru = torch.cat((embedded, context), dim=2)
        
        out, h = self.rnn(embeds, h)
        out = self.FC(out)

        return out, h

class Seq2Seq(torch.nn.Module):

    def __init__(self, units: int, n_tokens: int, n_stacks: int, bidirectional: bool, name: str):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(units, n_tokens, n_stacks, bidirectional, name+'_encoder').to('cuda')
        self.decoder = Decoder(units, n_tokens, n_stacks, bidirectional, name+'_decoder').to('cuda')

    def forward(self, text, target=None):
        enc_outs, enc_h = self.encoder(text)
        dec_out = self.decoder(enc_outs, enc_h, target)
        
        return dec_out

In [15]:
encoder = Encoder(32, tokenizer.vocab_size, 1, False, 'encoder').to('cuda')
decoder = Decoder(32, tokenizer.vocab_size, 1, False, 'decoder').to('cuda')
text = next(iter(data))[0][:4]
enc_outs, enc_h = encoder(text)
dec_out = decoder(enc_outs, enc_h)
dec_out.shape

torch.Size([4, 31, 50257])

In [18]:
model = Seq2Seq(32, tokenizer.vocab_size, 1, False, 'seq2seq')
model(text)[0].shape

torch.Size([31, 50257])

In [ ]:
def get_model(
    units: int,
    n_tokens: int,
    n_labels: int,
    n_stacks: int = 1,
    bidirectional: bool = False,
    name: str | None = None,
    cell_type: type[keras.layers.Layer] = keras.layers.LSTMCell
) -> keras.Model:
    '''Creates a model with RNN architecture for sequence to sequence classification.

    Arguments:
        units: dimensionality of RNN cells
        n_tokens: number of tokens in the tokenizer dictionary
        n_labels: number of labels to be predicted
        n_stacks: number of RNN cells in the stack (1 -- no stacking)
        bidirectional: whether or not the model is bidirectional
        name: the model name
        cell_type: type of a cell to use, either keras.layers.LSTMCell or keras.layers.GRUCell

    Returns:
        The model'''
    ...

Try to add attention to your model (for example [additive attention](https://keras.io/api/layers/attention_layers/additive_attention/)), does it perform better?

# Training

Train your model using teacher forcing. The idea is that the model predicts the next token that should follow, so one part of the model (called encoder) reads the text and output some state containing information about the text read. The other part of the model (called decoder) reads and already generated text (or in case of the teacher forcing the expected output) and predicts the next token for each one. 

In [19]:
def train(encoder, decoder, encoder_optimizer,
          decoder_optimizer, loss_f, dataloader, teacher = None):

    model.train()
    running_loss = 0
    
    for i, batch in enumerate(dataloader, start=1):
        inp, out_BOS, out_EOS, masking_PAD = batch
        decoder_optimizer.zero_grad()
        encoder_optimizer.zero_grad()
        enc_outs, enc_h = encoder(inp)
        seq_pred = decoder(enc_outs, enc_h, out_BOS if teacher else None)
        #seq_pred = model(inp, out_BOS if teacher else None) #out_EOS
        # loss = loss_f(seq_pred[masking_PAD], 
        #               out_EOS[masking_PAD])
        loss = loss_f(seq_pred.view(-1, seq_pred.size(-1)), 
                      out_EOS.view(-1))
        #return seq_pred, masking_PAD, out_EOS

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        running_loss += loss.item()
        if i % (len(dataloader) // 4) == 0:
            print(f'batch{i} cur_loss {running_loss / i}')
        
    return running_loss / len(dataloader)

In [20]:
# def evaluate(model, dataloader, tokenizer, metric, target = None):
#     model.to('cuda')
#     model.eval()
#     running_loss = 0
#     running_metric = 0
    
#     for i, batch in enumerate(dataloader, start=1):
#         inp, out_BOS, out_EOS, masking_PAD = batch
#         seq_pred = model(inp) #out_EOS

#         for seq, y_true, pad in zip(seq_pred, out_EOS, masking_PAD):
#             seq = seq.argmax(dim=-1)
#             candidates = tokenizer.decode(seq[pad])
#             references = torch.Tensor(tokenizer.decode(y_true[pad])).to('cuda')
#             metric.update(torch.Tensor("I wouldn't be able to solve the problem, even if I tried.").to('cuda'), [references])
        
#         loss = loss_f(seq_pred.view(-1, seq_pred.size(-1)), 
#                       out_EOS.view(-1))
#         running_loss += loss.item()
        
#         #if i % (len(dataloader) // 4) == 0:
#         #    print(f'batch{i} cur_loss {running_loss / i}')
#     print(metric.device)
#     return running_loss / len(dataloader), metric.compute()

In [21]:
%%time
EPOCHS = 20
#model = Seq2Seq(32, tokenizer.vocab_size, 3, True, 'seq2seq')
encoder = Encoder(32, tokenizer.vocab_size, 1, False, 'encoder').to('cuda')
decoder = Decoder(32, tokenizer.vocab_size, 1, False, 'decoder').to('cuda')
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)

loss_f = torch.nn.CrossEntropyLoss()
teacher = True
for epoch in range(EPOCHS):
    #metric = BLEUScore(n_gram=3, device='cuda')
    
    if epoch == 9:
        teacher = False
        print(f'Teacher forcing is OVER')
    loss = train(encoder, decoder, encoder_optimizer,
          decoder_optimizer, loss_f, data, teacher = teacher)
    print(loss)
#score = evaluate(model, data_test, tokenizer, metric)

batch859 cur_loss 2.392586445919432
batch1718 cur_loss 1.9209044858102056
batch2577 cur_loss 1.7363049594940583
batch3436 cur_loss 1.6277941676318577
1.6273915018755254
batch859 cur_loss 1.2576480487728008
batch1718 cur_loss 1.2407764716462013
batch2577 cur_loss 1.2280551945464668
batch3436 cur_loss 1.2183420067442727
1.2183333249693868
batch859 cur_loss 1.1718669483769897
batch1718 cur_loss 1.1651504768342718
batch2577 cur_loss 1.1579339662216226
batch3436 cur_loss 1.1538523602298307
1.153810159121533
batch859 cur_loss 1.1250854542563486
batch1718 cur_loss 1.119661390538543
batch2577 cur_loss 1.118158749475653
batch3436 cur_loss 1.1159583223122795
1.1160503899541556
batch859 cur_loss 1.0945259714987003
batch1718 cur_loss 1.0930846831712513
batch2577 cur_loss 1.0918675166116374
batch3436 cur_loss 1.0899569661778261
1.0898750732720817
batch859 cur_loss 1.073588047302366
batch1718 cur_loss 1.0725797521984757
batch2577 cur_loss 1.0709766620436887
batch3436 cur_loss 1.0705338169726282
1.07

In [60]:
test

tensor([[50256, 25443,   111, 12466, 16843,   121, 12466,    11, 16142, 43666,
         16843,   123,   140, 18849, 21727, 15166, 30143, 16843,   110, 12466,
         20375, 16843,   121, 12466, 40623, 22177, 16843,   120, 12466,    96,
           140, 50256],
        [50256,   229,   141, 21727,   220, 40623,   220,   116, 12466, 15166,
         20375,   220,    11, 38857, 18849, 30143, 20375, 21727, 16142,   229,
           141, 21727,   220, 45035, 20375,   220, 18849, 30143, 21727,   243,
           140, 50256]], device='cuda:0')

In [64]:
inp

tensor([[50256, 50256, 50256,  ...,   240,   140, 50256],
        [50256,   220, 16142,  ...,   107,   140, 50256],
        [50256, 16843,   114,  ...,   107,   140, 50256],
        ...,
        [50256, 50256, 50256,  ...,   255,   140, 50256],
        [50256, 18849,   123,  ...,   243,   140, 50256],
        [50256, 50256, 50256,  ...,   242,   140, 50256]], device='cuda:0')

In [88]:
inp, out_BOS, out_EOS, masking_PAD = next(iter(data))
enc_outs, enc_h = encoder(inp[:2])
print(tokenizer.decode(out_EOS[0][masking_PAD[0]]))
print(tokenizer.decode(decoder(enc_outs, enc_h)[0].argmax(dim=-1)[masking_PAD[0]]))

The policeman stood like a statue with his arms folded across his chest.
The police was the the the was..<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


# Testing

Make a function for text translation. Translate some text and evaluate model performance.

Take note that your model is set for training. During the inference process you will have to use parts of the model independently (including the RNN cells).

In [ ]:
def translate(
    text: str,
    tokenizer: Tokenizer,
    model: keras.Model,
    max_len: int = 20
) -> str:
    '''Predicts `text`translation using the `model`.

    Arguments:
        text: text to be translated
        tokenizer: tokenizer to use
        model: model ot use
        max_len: maximum length of the prediction (in tokens)

    Returns:
        tranlated text'''
    ...